In [1]:
import src.jss_graph_env.disjunctive_graph_jss_env as jss_env
import src.jsp_instance_parser 
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
import gym
from ray import tune
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel
from ray.rllib.models.catalog import ModelCatalog
from wrapper.jssp_wrapper import jssp_obs_wrapper

config = {
    "framework": "torch",
    "disable_env_checking":True,
    "num_workers"       : 0,
    "rollout_fragment_length": 50,
    "train_batch_size"  : 500,
    "sgd_minibatch_size": 64,
    "lr"                : 0.0001,
    "num_sgd_iter"      : 1,
    "mcts_config"       : {
        "puct_coefficient"   : 1.5,
        "num_simulations"    : 100,
        "temperature"        : 1.0,
        "dirichlet_epsilon"  : 0.20,
        "dirichlet_noise"    : 0.03,
        "argmax_tree_policy" : False,
        "add_dirichlet_noise": True,
    },
    "ranked_rewards"    : {
        "enable": False,
    },
    "model"             : {
        "custom_model": "dense_model",

    },
}

In [2]:

def env_creator(env_config):
    #env = discretetobox(gym.make("Taxi-v3"))
    #env = gym.make('LunarLander-v2')
    path='resources/jsp_instances/standard/abz8.txt'
    curr_instance=src.jsp_instance_parser.parse_jps_standard_specification(path)
    res,std_matrix=curr_instance
    env = jssp_obs_wrapper(jss_env.DisjunctiveGraphJssEnv(res,default_visualisations='gantt_console'))
    return env
ModelCatalog.register_custom_model("dense_model", DenseModel)    
tune.register_env('customjssp',env_creator)

agent = AlphaZeroTrainer( config=config, env='customjssp')

2022-08-25 12:17:51,151	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-08-25 12:17:51,159	INFO trainer.py:903 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=15864) [12:18:08] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'    
(RolloutWorker pid=15864)            INFO     instance appears to contain 20 jobs, 15 machines            
(RolloutWorker pid=15863) [12:18:08] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'    
(RolloutWorker pid=15863)            INFO     instance appears to contain 20 jobs, 15 machines            
(RolloutWorker pid=15862) [12:18:08] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'    
(RolloutWorker pid=15862)            INFO     instance appears to contain 20 jobs, 15 machines            
(RolloutWorker pid=15866) [12:18:08] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'    
(RolloutWorker pid=15866)            INFO     instance appears to contain 20 jobs, 15 machines            
(RolloutWorker pid=15867) [12:18:08] INFO     parsing file 'resources/jsp_instances/standard/abz8.txt'    
(RolloutWorker pid=15867)            

(RolloutWorker pid=15864) 2022-08-25 12:18:08,116	ERROR worker.py:451 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=15864, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f9870114160>)
(RolloutWorker pid=15864)   File "/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 630, in __init__
(RolloutWorker pid=15864)     self._build_policy_map(
(RolloutWorker pid=15864)   File "/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1788, in _build_policy_map
(RolloutWorker pid=15864)     self.policy_map.create_policy(
(RolloutWorker pid=15864)   File "/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/policy/policy_map.py", line 152, in create_policy
(RolloutWorker pid=15864)     self[policy_id] = class_(observation_sp

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=15862, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7fecb694c1f0>)
  File "/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 630, in __init__
    self._build_policy_map(
  File "/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1788, in _build_policy_map
    self.policy_map.create_policy(
  File "/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/policy/policy_map.py", line 152, in create_policy
    self[policy_id] = class_(observation_space, action_space, merged_config)
  File "/Users/felix/miniconda3/envs/customjssp/lib/python3.10/site-packages/ray/rllib/contrib/alpha_zero/core/alpha_zero_trainer.py", line 150, in __init__
    model = ModelCatalog.get_model_v2(
  File "/

In [ ]:
print("start training")
for _ in range(0,10):
    agent.train()
    print(f"training iteration {_} finished")
    #agent.save(f"save_az/rllib_checkpoint{_}")
    agent.save_checkpoint(f"training_checkpoints/checkpoints_az_jsp")
    #agent.save_to_object(f"objects_az/rllib_checkpoint{_}")

In [ ]:
import numpy as np
import random
env.reset()
action_space=env.action_space
action_mask=env.valid_action_mask()
#task_mask = self.valid_action_mask
#job_mask = np.array_split(action_mask, 10)[action]
action_list=[]

action_list=np.arange(0,env.n_jobs*env.n_machines)


for i in range(1000):
    action_mask=env.valid_action_mask()
    actions=action_list[action_mask]
    next_action=random.choice(actions)
    #next_action=actions[action_mask[0]]
    #print(next_action)
    state,reward,done,info =env.step(next_action)
    if done == True:
        print(f"finished after {i} steps")
        break
    #env.render()

env.render()

In [ ]:
env.render()

In [ ]:
import src.modelbased.alphazero as az

In [ ]:
env.close()